# Assumptions
Group n-grams if in dependency tree they are either
    1. Direct children with only one link between them or
    2. Leafs on the same level with the same parent

In [453]:
import spacy
from nltk import Tree
from typing import List
nlp = spacy.load('en')
stopword = "the in has be".split()

class Node:
    def __init__(self):
        self.data = []
        self.children = []

In [454]:
def pos_n_gram(sentence: str, keep_stop_words: bool=False) -> List[str]:
    """
    Generate an n-gram based on the POS tagged dependency tree of the sentence that is "simplified" down according
    to a few assumptions that dictate a good sentence split. These assumptions are as follows:
        1. If two words are leafs and on the same level with the same parent they can be grouped as an n-gram
        2. If there is a sequence of parent-child relationships with only 1 child they can be grouped as one
           n-gram
           
    
    """
    pos_tagged_n_grams = []
    
    def to_nltk_tree(node):
        current_node = node
        backlog = []
        while current_node.n_lefts + current_node.n_rights == 1:
            backlog.append((current_node.orth_, current_node.i))
            current_node = list(current_node.children)[0]

        backlog.append((current_node.orth_, current_node.i))
        if current_node.n_lefts + current_node.n_rights > 1:
            good_children = [child for child in current_node.children if len(list(child.children)) > 0]
            bad_children = [(child.orth_, child.i) for child in current_node.children if child not in good_children]
            pos_tagged_n_grams.append(backlog)
            pos_tagged_n_grams.append(bad_children)
            return Tree(backlog, [Tree(bad_children, [])] + [to_nltk_tree(child) for child in good_children])
        else:
            pos_tagged_n_grams.append(backlog)
            return Tree(backlog, [])
        
    def strip_nothing_unigrams(n_grams):
        return [n_gram for n_gram in n_grams if not (len(n_gram.split(" ")) == 1 and n_gram.split(" ")[0] in stopword)]

    query = " ".join([word for word in sentence.split() if word not in stopword or keep_stop_words])
    doc = nlp(query)
    to_nltk_tree(list(doc.sents)[0].root);
    # print(nltk_tree)

    sort_inner = [sorted(nltk_child, key=lambda x: x[1]) for nltk_child in pos_tagged_n_grams]

    nltk_averages = []
    for nltk_child in sort_inner:
        nltk_averages.append((nltk_child, max(x[1] for x in nltk_child)))

    sorted_outer = list(sorted(nltk_averages, key=lambda x: x[1]))

    n_grams = []
    for nltk_average in sorted_outer:
        n_grams.append(" ".join(word[0] for word in nltk_average[0]))
        
    return n_grams
        
pos_n_gram("Matt has already completed the homework for the class", True)

['Matt has already', 'completed', 'the', 'homework', 'for the class']

In [237]:
query = u''
pp(query)
query = " ".join([word for word in query.split() if word not in stopword])
pp(query)
doc = nlp(query)
nltk_tree = to_nltk_tree(list(doc.sents)[0].root)
print(nltk_tree.leaves() + [nltk_tree.label()])

         completed                   
  ___________|____________            
 |    |      |         homework      
 |    |      |       _____|_______    
 |    |      |      |            for 
 |    |      |      |             |   
 |    |      |      |           class
 |    |      |      |             |   
Matt has  already  the           the 

     completed         
  _______|________      
 |       |     homework
 |       |        |     
 |       |       for   
 |       |        |     
Matt  already   class  

completed
["[('class', 5), ('homework', 3), ('for', 4)]", "[('completed', 2)]"]


In [370]:
def pp(query):
    doc = nlp(query)
    [to_nltk_tree(sent.root).pretty_print() for sent in doc.sents]

In [433]:
query = u'Matt finished the homework before class even started'
# pp(query)
query = " ".join([word for word in query.split() if word not in stopword])
doc = nlp(query)
nltk_tree = to_tree(list(doc.sents)[0].root);
nltk_averages = []
# print(nltk_tree)

sort_inner = [sorted(nltk_child, key=lambda x: x[1]) for nltk_child in nltk_tree]

for nltk_child in sort_inner:
    nltk_averages.append((nltk_child, max(x[1] for x in nltk_child)))


sorted_outer = list(sorted(nltk_averages, key=lambda x: x[1]))
[print(x) for x in sorted_outer]



n_grams = []
for nltk_average in sorted_outer:
    n_grams.append(" ".join(word[0] for word in nltk_average[0]))

n_grams

([('finished', 1)], 1)
([('Matt', 0), ('homework', 2)], 2)
([('before', 3), ('class', 4), ('even', 5)], 5)
([('started', 6)], 6)


['finished', 'Matt homework', 'before class even', 'started']

In [347]:
nltk_tree.leaves()

<bound method Tree.leaves of Tree([('is', 2)], [Tree([], []), Tree([('defense', 1), ('Unilateral', 0)], []), Tree([('idea', 5)], [Tree([('a', 3), ('bad', 4)], [])])])>